# GPUコンピューティングを使ったデータ分析

## Numbaを使ってGPUコードを生成する

In [1]:
!numba -s

System info:
--------------------------------------------------------------------------------
__Time Stamp__
Report started (local time)                   : 2025-05-03 23:49:08.997401
UTC start time                                : 2025-05-03 23:49:08.997406
Running time (s)                              : 1.541703

__Hardware Information__
Machine                                       : x86_64
CPU Name                                      : skylake-avx512
CPU Count                                     : 4
Number of accessible CPUs                     : 4
List of accessible CPUs cores                 : 0 1 2 3
CFS Restrictions (CPUs worth of runtime)      : None

CPU Features                                  : 64bit adx aes avx avx2 avx512bw
                                                avx512cd avx512dq avx512f avx512vl
                                                bmi bmi2 clflushopt clwb cmov
                                                crc32 cx16 cx8 f16c fma fsgsbase
        

In [2]:
from numba import jit, prange
import numpy as np
from PIL import Image


def compute_point(c, max_iter=200):
    i = -1
    z = complex(0, 0)
    while abs(z) < 2:
        i += 1
        if i == max_iter:
            break
        z = z**2 + c
    return 255 - (255 * i) // max_iter


compute_point_numba = jit()(compute_point)
compute_point_numba_forceobj = jit(forceobj=True)(compute_point)
compute_point_numba(complex(4, 4))
compute_point_numba_forceobj(complex(4, 4))

size = 2000
start = -1.5, -1.3
end = 0.5, 1.3

img_array = np.empty((size, size), dtype=np.uint8)


def do_all(size, start, end, img_array, compute_fun):
    startx, starty = start
    endx, endy = end
    for xp in range(size):
        x = (endx - startx)*(xp/size) + startx  # precision issues
        # x = (xp - size/2) / (size/4)   # precision issues
        # print(x)
        for yp in range(size):
            y = (endy - starty)*(yp/size) + starty  # precision issues
            img_array[yp, xp] = compute_fun(complex(x,y))


do_all(size, start, end, img_array, compute_point_numba)
img = Image.fromarray(img_array, mode="P")
img.save("mandelbrot.png")
# img.putpalette(ImagePalette.sepia())

# compute_point_typed = jit(compute_point, "uint8(complex128)", nopython=True)


@jit(nopython=True, parallel=True, nogil=True)
def pdo_all(size, start, end, img_array, compute_fun):
    startx, starty = start
    endx, endy = end
    for xp in prange(size):
        x = (endx - startx)*(xp/size) + startx  # precision issues
        # x = (xp - size/2) / (size/4)   # precision issues
        # print(x)
        for yp in range(size):  # put prange here?
            # Loops are fine with Numba
            y = (endy - starty)*(yp/size) + starty  # precision issues
            b = complex(0, 0)
            b = compute_fun(complex(x, y))
            img_array[yp, xp] = b


# parallel_diagnostics - just note

# print(threading_layer())

In [3]:
from math import ceil
from numba import cuda
import numpy as np
from PIL import Image

size = 2000
start = -1.5, -1.3
end = 0.5, 1.3


@cuda.jit(device=True)
def compute_point(c):
    i = -1
    z = complex(0, 0)
    while abs(z) < 2:
        i += 1
        if i == 255:
            break
        z = z**2 + c
    return 255 - (255 * i)


@cuda.jit
def compute_all_points_doesnt_work(start, end, size, img_array):
    x, y = cuda.grid(2)
    if x >= img_array.shape[0] or y >= img_array.shape[1]:
        return
    mandel_x = (end[0] - start[0])*(x/size) + start[0]
    mandel_y = (end[1] - start[1])*(y/size) + start[1]
    img_array[y, x] = compute_point(complex(mandel_x, mandel_y))


@cuda.jit
def compute_all_points(startx, starty, endx, endy, size, img_array):
    x, y = cuda.grid(2)
    if x >= img_array.shape[0] or y >= img_array.shape[1]:
        return
    mandel_x = (end[0] - startx)*(x/size) + startx
    mandel_y = (end[1] - starty)*(y/size) + starty
    img_array[y, x] = compute_point(complex(mandel_x, mandel_y))


img_array = np.empty((size, size), dtype=np.uint8)
threads_per_block_2d = 16, 16
blocks_per_grid_2d = ceil(size / 16), ceil(size / 16)

compute_all_points[blocks_per_grid_2d, threads_per_block_2d](start[0], start[1], end[0], end[1], size, img_array)

img = Image.fromarray(img_array, mode="P")
img.save("mandelbrot.png")

/usr/local/lib/python3.11/dist-packages/numba_cuda/numba/cuda/cudadrv/devicearray.py:890: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [4]:
from numba import vectorize, cuda
import numpy as np
from PIL import ImagePalette, Image

size = 2000
start = -1.5, -1.3
end = 0.5, 1.3


def compute_point_255_fn(c):
    i = -1
    z = complex(0, 0)
    while abs(z) < 2:
        i += 1
        if i == 255:
            break
        z = z**2 + c
    return 255 - (255 * i) // 255


compute_point_vectorized = vectorize(["uint8(complex128)"], target="cuda")(compute_point_255_fn)


def prepare_pos_array(start, end, pos_array):
    size = pos_array.shape[0]
    startx, starty = start
    endx, endy = end
    for xp in range(size):
        x = (endx - startx)*(xp/size) + startx
        for yp in range(size):
            y = (endy - starty)*(yp/size) + starty 
            pos_array[yp, xp] = complex(x, y)


pos_array = np.empty((size, size), dtype=np.complex128)

img_array = np.empty((size, size), dtype=np.uint8)
compute_point_vectorized(pos_array)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

## GPUコードのパフォーマンス分析

In [5]:
import numpy as np
import cupy as cp
from cupyx.time import repeat

size = 5000

my_matrix = cp.ones((size, size), dtype=cp.uint8)
print(type(my_matrix))
np_matrix = my_matrix.get()
print(type(np_matrix))  # can be sent to numpy dependent libraries

2 * my_matrix

2 * np_matrix

print(repeat(lambda : 2 * my_matrix, n_repeat=200))

<class 'cupy.ndarray'>
<class 'numpy.ndarray'>
<lambda>            :    CPU:    27.447 us   +/-  7.770 (min:    23.265 / max:    92.678) us     GPU-0:   732.610 us   +/- 38.688 (min:   677.856 / max:   822.624) us


/usr/local/lib/python3.11/dist-packages/cupyx/time.py:62: UserWarning: cupyx.time.repeat has been moved to cupyx.profiler.benchmark since CuPy v10. Access through cupyx.time is deprecated and will be removed in the future.
  _warnings.warn(


In [6]:
%%time
from math import ceil

import numpy as np
import cupy as cp
from cupyx.time import repeat
from numba import cuda
from PIL import Image

size = 2000
start = -1.5, -1.3
end = 0.5, 1.3


@cuda.jit
def compute_all_mandelbrot(startx, starty, endx, endy, size, img_array):
    x, y = cuda.grid(2)
    if x >= img_array.shape[0] or y >= img_array.shape[1]:
        return
    mandel_x = (end[0] - startx)*(x/size) + startx
    mandel_y = (end[1] - starty)*(y/size) + starty
    c = complex(mandel_x, mandel_y)
    i = -1
    z = complex(0, 0)
    while abs(z) < 2:
        i += 1
        if i == 255:
            break
        z = z**2 + c
    img_array[y, x] = i


threads_per_block_2d = 16, 16
blocks_per_grid_2d = ceil(size / 16), ceil(size / 16)

cp_img_array = cp.empty((size, size), dtype=cp.uint8)

compute_all_mandelbrot[blocks_per_grid_2d, threads_per_block_2d](
    start[0], start[1],
    end[0], end[1],
    size, cp_img_array)

# print(repeat(
#     lambda: compute_all_mandelbrot[blocks_per_grid_2d, threads_per_block_2d](
#         start[0], start[1], end[0], end[1], size, cp_img_array),
#     n_repeat=200))

img = Image.fromarray(cp.asnumpy(cp_img_array), mode="P")
img.save("imandelbrot.png")

CPU times: user 994 ms, sys: 7.97 ms, total: 1 s
Wall time: 1 s


In [7]:
%%time
from math import ceil

import numpy as np
import cupy as cp
from cupyx.time import repeat
from PIL import Image

size = 2000
start = -1.5, -1.3
end = 0.5, 1.3

cp_img_array = cp.empty((size, size), dtype=cp.uint8)


def prepare_pos_array(start, end, pos_array):
    size = pos_array.shape[0]
    startx, starty = start
    endx, endy = end
    for xp in range(size):
        x = (endx - startx)*(xp/size) + startx
        for yp in range(size):
            y = (endy - starty)*(yp/size) + starty 
            pos_array[yp, xp] = complex(x, y)


pos_array = np.empty((size, size), dtype=np.complex64)
prepare_pos_array(start, end, pos_array)

cp_pos_array = cp.array(pos_array)

threads_per_block = 16 ** 2
blocks_per_grid = ceil(size / 16) ** 2

c_compute_mandelbrot = cp.RawKernel(r'''
#include <cupy/complex.cuh>
extern "C" __global__
void raw_mandelbrot(const complex<float>* pos_array,
             char* img_array) {
    int x = blockDim.x * blockIdx.x + threadIdx.x;
    int i = -1;
    complex<float> z = complex<float>(0.0, 0.0);
    complex<float> c = pos_array[x];
    while (abs(z) < 2) {
        i++;
        if (i == 255) break;
        z = z*z + c;
    }
    img_array[x] = i;
}
''', 'raw_mandelbrot')
c_compute_mandelbrot((blocks_per_grid,) , (threads_per_block,), (cp_pos_array, cp_img_array))
img = Image.fromarray(cp.asnumpy(cp_img_array), mode="P")
img.save("cmandelbrot.png")


# print(repeat(
#     lambda: c_compute_mandelbrot((blocks_per_grid,) , (threads_per_block,), (cp_pos_array, cp_img_array)),
#     n_repeat=200))

CPU times: user 1.19 s, sys: 28 ms, total: 1.22 s
Wall time: 1.21 s
